In [1]:
import numpy as np
import csv
import os
from collections import defaultdict

In [2]:
files = os.listdir("./DATA/np_data")[1:500]
uids = list(set([fname.split('.')[0] for fname in files]))

#### Конвертация монофонных выравниваний в трифонные

Сначала нужно отрезать от фонем суффиксы \__B_, \__I_, \__E_, \__S_, потому что они означают _начало_, _конец_ и т.д., и  соответственно, они нам не очень нужны. <br>
И сразу же делаем словарь, где в качестве ключа выступает индекс (или номер) фонемы согласно файлу, а в качестве значения сама фонема.

In [3]:
dictionary = {}
with open('./DATA/data/lang/phones.txt') as fin:
    reader = csv.reader(fin, delimiter=' ')
    for row in reader:
        dictionary[int(row[1])] = row[0].split('_')[0]

Чтобы конвертировать монофоны в трифоны, нужно сначала сгруппировать фонемы, потом найти уникальные группы в группах, создать трифоны из этих групп и заменить номера фонем самими фонемами. Для первых фонем и последних трифоны создавать не нужно, заменим их просто на SIL.

In [4]:
def convert_to_triphones(uid):
    targs = np.load('./DATA/np_data/' + uid + '.targs.npy')
    phones = []
    temp = []
    for phone in targs:
        if temp and phone not in temp:
            phones.append(temp)
            temp = []
        temp.append(phone)
    
    phones.append(temp)
    
    unique = [x[0] for x in phones]
    triphones = [unique[i:i+3] for i in range(len(unique)-3+1)]
    #print(triphones)
    result = []
    result += phones.pop(0)

    for i in range(len(phones)-1):
        result += ['_'.join([dictionary[x] for x in triphones[i]]) for _ in range(len(phones[i]))]
    result += phones[-1]
    
    for n, i in enumerate(result):
        if i == 1:
            result[n] = 'SIL'
            
    return result

Загружаем вектора с признаками.

In [5]:
def load_vectors(uid):
    feats = np.load('DATA/np_data/' + uid + '.feats.npy')
    return feats

Загружаем каждый файл, делаем конфертацию и загрузку признаков.

In [6]:
def make_triphones_vectors():
    triphones_vectors = defaultdict(list)
    
    for uid in uids:
        triphones = convert_to_triphones(uid)
        vectors = load_vectors(uid)

        for i in range(len(triphones)):
            triphones_vectors[triphones[i]].append(vectors[i])
    
    return triphones_vectors

In [7]:
triphs_vecs = make_triphones_vectors()

#### Оценка параметров

Теперь для каждого трифона нужно оценить mean и std. Конкатенируем их.

In [8]:
triphones_estimation = {}
for triph in triphs_vecs.keys():
    mean = np.mean(triphs_vecs[triph], axis=0)
    std = np.std(triphs_vecs[triph], axis=0)
    triphones_estimation[triph] = np.append(mean, std, axis=0)

In [9]:
triphs = list(triphones_estimation.keys())

#### K-means

Для кластеризации выбираем трифоны с общей фонемой посередине. Пусть общей фонемой будет <b>AH0</b>. По IPA это <b>ə</b>. Примеры употребления: _sofa_ (S OW1 F AH0), _alone_ (AH0 L OW1 N).

In [10]:
AH_triphones = []
for x in triphs:
    if "_" in x:
        if x.split("_")[1] == "AH0":
            AH_triphones.append(x)

In [11]:
from sklearn.cluster import KMeans

In [12]:
X = np.array([triphones_estimation[AH_triphones[i]] for i in range(len(AH_triphones))])

Запускаем функцию для кластеризации.

In [13]:
kmeans = KMeans(10)
kmeans.fit(X)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=10, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [14]:
clusters = defaultdict(list)
for i in list(zip(kmeans.labels_, AH_triphones)):
    clusters[i[0]].append(i[1])

In [15]:
for i in clusters:
    print("Cluster {}:".format(i), clusters[i], "\n")

Cluster 0: ['D_AH0_SIL', 'D_AH0_HH', 'TH_AH0_HH', 'NG_AH0_K', 'M_AH0_SIL', 'IY1_AH0_N', 'R_AH0_SIL', 'K_AH0_SIL', 'N_AH0_EU0', 'R_AH0_R', 'M_AH0_AO1', 'ZH_AH0_SIL', 'IY0_AH0_SIL'] 

Cluster 1: ['F_AH0_B', 'T_AH0_W', 'CH_AH0_L', 'S_AH0_L', 'K_AH0_L', 'T_AH0_L', 'Z_AH0_L', 'F_AH0_L', 'P_AH0_L', 'B_AH0_L', 'S_AH0_R', 'DH_AH0_W', 'SH_AH0_L', 'T_AH0_R', 'Z_AH0_B', 'V_AH0_L', 'G_AH0_L', 'D_AH0_L', 'JH_AH0_L', 'G_AH0_B'] 

Cluster 2: ['P_AH0_S', 'NG_AH0_S', 'L_AH0_S', 'DH_AH0_CH', 'S_AH0_TH', 'UW0_AH0_S', 'DH_AH0_F', 'S_AH0_Z', 'D_AH0_F', 'D_AH0_TH', 'TH_AH0_S', 'Z_AH0_F', 'V_AH0_S', 'S_AH0_F', 'T_AH0_F', 'EY1_AH0_S', 'R_AH0_S', 'DH_AH0_SIL', 'S_AH0_S', 'K_AH0_S', 'AY1_AH0_S', 'Z_AH0_Z', 'T_AH0_CH', 'IY0_AH0_S', 'DH_AH0_TH', 'SH_AH0_S', 'NG_AH0_F', 'Z_AH0_S', 'IY1_AH0_F', 'L_AH0_TH', 'D_AH0_S', 'HH_AH0_Z', 'W_AH0_Z', 'G_AH0_S', 'F_AH0_S', 'K_AH0_Z', 'N_AH0_F', 'T_AH0_T', 'W_AH0_S', 'T_AH0_S', 'UW1_AH0_F', 'N_AH0_S', 'JH_AH0_S', 'M_AH0_S', 'CH_AH0_F', 'OY1_AH0_S', 'DH_AH0_S'] 

Cluster 3: ['P_

AH0 (ə) представляет собой результат редукции, что означает ослабление гласных в безударном положении. Ноль в обозначении фонемы AH0 (ə) означает безударный звук. <br>
Кластер #0 достаточно хороший. Фонема часто стоит в конце слова (на письме отображается диграфами or/er/r). Т.е. в целом понятно, по какому принципу сгруппированы трифоны.<br>
Тоже самое можно сказать и про кластер #7, потому что в безударном положении в начале слова буквы a/o тоже передаются звуком AH0 (ə). <br>
Остальные кластеры сложно проверить "на глаз", потому что звук AH0 (ə) встречается в безударном слоге в середине большого числа слов, и особого правила для этого не существует.<br>
В кластере #2 в большинстве трифонов за AH0 (ə) следует или фрикативные (щелевые) согласные F,S,Z,TH или аффрикат CH.<br>
В кластере #3 за звуком AH0 (ə) идут как взрывные согласные, так и носовые сонанты.<br>
В кластере #6 за фонемой следуют только AH0 (ə) только взрывные согласные.<br>
Кластеры #4, #5, #7, #8 и #9 включают себе разные группы фонем, которые следуют после AH0 (ə). Поэтому не совсем явно, по какому принципу трифоны оказались в рдном кластере.<br>